In [4]:
import numpy as np
import pandas as pd
import NegativeClassOptimization.config as config
import glob
from pathlib import Path
import re
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'NegativeClassOptimization'

In [1]:
config.antigens

NameError: name 'config' is not defined

In [ ]:
glob.glob(./torch_model/Frozen_MiniAbsolut_ML/hifh_vs_low/)